In [4]:
import numpy as np
import os
from DH_dynamics import dh_robot_config
# from JS_control import cacl_torque
import matplotlib.pyplot as plt
from pyrep import PyRep
from arm import CtRobot
from pyrep.backend import vrep
import sympy as sp
from pyrep.const import JointMode
import time

%matplotlib notebook

pr = PyRep()

In [5]:
import numpy as np

def cacl_torque(robot, kv, kp, posd, posm, veld, velm, xyz, accd=None):
    '''
    robot: Class that contains dynamic information of robot
    posd: numpy array of dimension 6 representing desired position and orientation of joint
    posm: numpy array of dimension 6 representing desired position and orientation of joint
    veld: numpy array of dimension 6 representing measured velocity of joint
    velm: numpy array of dimension 6 representing measured velocity of joint
    '''
    num_joints = robot.num_joints
#     M = np.array(robot._Mq[0](*tuple(np.concatenate((posm, xyz)))))
    M = np.eye(num_joints)
    G = np.array(robot._Gq[0](*tuple(np.concatenate((posm, xyz))))).reshape(num_joints, 1)
#     G = np.zeros((7,1))
    gain_v = kv.dot((veld-velm).reshape(num_joints, 1))
    gain_x = kp.dot((posd-posm).reshape(num_joints, 1))
    if accd is not None:
        tau = G+M.dot((gain_v + gain_x +accd.reshape(num_joints, 1)))
    else:
        tau = G+M.dot((gain_v + gain_x))
    return tau.reshape(num_joints)


In [6]:
param = ['D', 'a', 'alpha', 'theta', 'num_joints', 'jointType', 'Tbase', 'L', 'M']
config = dict()
for i in range(len(param)):
    config[param[i]] = np.load('./robot_config/config_ct_7DOF/%s.npy'%param[i])

In [7]:
robot = dh_robot_config(int(config['num_joints']), config['alpha'], config['theta'], config['D'], config['a'], 
                                                config['jointType'], config['Tbase'], config['L'], config['M'])
robot.initKinematicTransforms()

Calculating link 0
Calculating link 1
Calculating link 2
Calculating link 3
Calculating link 4
Calculating link 5
Calculating link 6
Calculating link 7
Calculating Mass and Gravity Matrix...
-------------------------------------------------------------------
Calculation complete


In [26]:
def Gq(q, x=[0, 0, 0]):
    G = np.zeros((robot.num_joints, 1))
    q.extend(x)
#     print(q, type(q))
    for ii in range(robot.num_joints):            
        G += robot._Gq[ii](*tuple(q))
    return G
#     return np.sum(np.array([robot._Gq[i](*tuple(q)) for i in range(robot.num_joints)]),axis=1)

In [27]:
def Mq(q, x=[0, 0, 0]):
    M = np.zeros((robot.num_joints, robot.num_joints))
    q.extend(x)
    for ii in range(robot.num_links):            
        M += robot._Mq[ii](*tuple(q))
    return M

In [28]:
ts = time.time()
for i in range(100):
    Gq([0,0,0,0,0,0,0])
print((time.time()-ts)/100)

0.0015024280548095704


In [8]:
robot._Gq[1](0,0,0,0,0,0,0,0,0,0).shape

(7, 1)

In [ ]:
robot._Gq

In [ ]:
import cloudpickle

G = cloudpickle.load(open('./%s/Gq' % 'robot_config', 'rb'))
G

In [ ]:
robot._Gq[0](0,0,0,0,0,0,0,0,0,0)

## Simulation Set up

### For all joints

In [ ]:
pr.launch(os.getcwd() + '/Modified_DH_ct_7DOF.ttt', headless=False)
ct_robot = CtRobot()

xyz = np.array([0, 0, 0])

In [ ]:
dt = 0.01
pr.set_simulation_timestep(dt)

In [ ]:
# Starting simulation
pr.start()
err_pos = []
err_vel = []
mespos = []
depos = []

# Set up joints to be in force control mode
for i in range(ct_robot._num_joints):
    ct_robot.joints[i].set_joint_mode(JointMode.FORCE)
    ct_robot.joints[i].set_control_loop_enabled(False)
    ct_robot.joints[i].set_motor_locked_at_zero_velocity(True)
for j in range(1, ct_robot._num_joints + 1):
    ct_robot.arms[i].set_dynamic(True)
ct_robot.arms[0].set_dynamic(False)
pr.step()

# Desired trajectory
end_time = 4
t_dic = np.linspace(0, end_time, num=int(end_time/dt), endpoint=True)
t = sp.Symbol('t')
# disired trajectory of position for each joint

# # Sine wave activation
# traj = [
#     0.0*sp.sin(t*4),
#     0.0*sp.cos(t*4),
#     (0/180*np.pi)*sp.sin(t*np.pi),
#     (0/180*np.pi)*sp.sin(t*4),
#     (30/180*np.pi)*sp.cos(t*4),
#     (0/180*np.pi)*sp.cos(t*4),
#     0.00*sp.sin(t/2)
# ]

# Step response
traj = [
    0.000 * sp.ones(1),
    0.000 * sp.ones(1),
    (0/180*np.pi)*sp.ones(1),
    (0/180*np.pi)*sp.ones(1),
    (0/180*np.pi)*sp.ones(1),
    (0/180*np.pi)*sp.ones(1),
    0.00*sp.ones(1)
]

pos = [sp.lambdify(t, i) for i in traj]
vel = [sp.lambdify(t, i.diff(t)) for i in traj]
acc = [sp.lambdify(t, i.diff(t).diff(t)) for i in traj]

# Gain
# kp = np.diag([100, 100, 100, 100, 100, 100, 100])
# kv = np.diag([0, 0, 0, 0, 0, 0, 0])
kp = np.eye(7)*0.6
kv = 6*np.sqrt(kp)

# Simulation
for i in range(int(end_time/dt)):
#     ts = time.time()
    posd = np.array([j(t_dic[i]) for j in pos]).squeeze()
    veld = np.array([j(t_dic[i]) for j in vel]).squeeze()
    accd = np.array([j(t_dic[i]) for j in acc]).squeeze()
    posm = np.array(ct_robot.get_joint_positions())
    velm = np.array(ct_robot.get_joint_velocities())
    tau = cacl_torque(robot, kp, kv, posd, posm, veld, velm, xyz)
    # print('Time for computing torque: %.4f'%(time.time()-ts))
#     tau[0:2] = 0
    print(tau)
    ct_robot.set_joint_target_velocities((np.sign(tau)*1000000).tolist())
    ct_robot.set_joint_forces(np.abs(tau).tolist())
    pr.step()
#     ts = time.time()
    measure_pos = np.array(ct_robot.get_joint_positions())
    measure_vel = np.array(ct_robot.get_joint_velocities())
#     print('Time for Interacting with V-REP: %.4f'%(time.time()-ts))
    
    # Record error of each joint
    err_pos.append(posd - measure_pos)
    err_vel.append(veld - measure_vel)
    depos.append(posd)
    mespos.append(measure_pos)


In [ ]:
# Change Unit
err_pos = np.array(err_pos)
mespos = np.array(mespos)
depos = np.array(depos)

for name in ['err_pos', 'mespos', 'depos']:
    x = eval(name)
    x[:, 0:1] = x[:, 0:1]*1000
    x[:, 1:2] = x[:, 1:2]*1000
    x[:, 6:7] = x[:, 6:7]*1000
    x[:, 2:3] = x[:, 2:3]*180/np.pi
    x[:, 3:4] = x[:, 3:4]*180/np.pi
    x[:, 4:5] = x[:, 4:5]*180/np.pi
    x[:, 5:6] = x[:, 5:6]*180/np.pi

In [ ]:
# Plot error of position from joint to joint
plt.figure(figsize=(18,10))
plt.suptitle('Error')
for i in range(7):
    plt.subplot(2,4,i+1)
    plt.title('Joint%d'%(i+1))
    plt.plot(np.array(err_pos)[:, i])
    if i in [0,1,6]:
        plt.ylabel('mm')
    else:
        plt.ylabel('deg')

In [ ]:
# Plot error of position from joint to joint
plt.figure(figsize=(18,10))
plt.suptitle('Absoluter value')
for i in range(7):
    plt.subplot(2,4,i+1)
    plt.title('Joint%d'%(i+1))
    plt.plot(mespos[:, i], label='mesaured')
    plt.plot(depos[:, i], label='desired')
    plt.legend()
    if i in [0,1,6]:
        plt.ylabel('mm')
    else:
        plt.ylabel('deg')


In [ ]:
pr.stop()

In [ ]:
pr.shutdown()